In [35]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from astropy.io import fits, ascii
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy import units as u

from IPython.display import Image
from skimage.io import imread

from toasty import toast, cartesian_sampler, normalizer

from collections import namedtuple

import ps1skycell_toast as pssc

import time

In [2]:
def minmax(arr):
    return [min(arr),max(arr)]

In [ ]:
def M101Sampler():
    
    def vec2Pix(raArr,decArr):
        
        tile = np.zeros(raArr.shape)
        
        raArr[raArr < 0] += 2 * np.pi # making all ras 0 - 2pi
        
        raCorners = np.array([raArr[0][0],raArr[0][-1],raArr[-1][0],raArr[-1][-1]])
        decCorners = np.array([decArr[0][0],decArr[0][-1],decArr[-1][0],decArr[-1][-1]])       
        minRa = min(raCorners)
        maxRa = max(raCorners)
        minDec = min(decCorners)
        maxDec = max(decCorners)

        
        #   THIS IS ALL TEMPORARY 
        m101Pos = SkyCoord(210.79995, 54.34410, frame='icrs', unit='deg')
        if ((m101Pos.ra.rad >= minRa) and (m101Pos.ra.rad <= maxRa)):
            if ((m101Pos.dec.rad >= minDec) and (m101Pos.dec.rad <= maxDec)):
   
                fitsFile = fits.open('rings.v3.skycell.2381.053.stk.r.unconv.fits')
                rfWcs = wcs.WCS(fitsFile[1].header)
                footprint=np.array([SkyCoord(*x,frame='icrs',unit='deg') for x in rfWcs.calc_footprint(fitsFile[1].header) ])
                imgData = fitsFile[1].data
                fitsFile.close()
                
                fpMinRa = min(x.ra.rad for x in footprint)#(min(footprint[:,0])*u.deg).to(u.rad)
                fpMaxRa = max(x.ra.rad for x in footprint)#(max(footprint[:,0])*u.deg).to(u.rad)
                fpMinDec = min(x.dec.rad for x in footprint)#(min(footprint[:,1])*u.deg).to(u.rad)
                fpMaxDec = max(x.dec.rad for x in footprint)#(max(footprint[:,1])*u.deg).to(u.rad)

                print("woot")
                fpMask = (raArr <= fpMaxRa)  & (raArr >= fpMinRa) & (decArr <= fpMaxDec) & (decArr >= fpMinDec)
                print(np.sum(fpMask))
                print(fpMinRa,fpMaxRa)
                print(fpMinDec,fpMaxDec)
                print()
                
                ny, nx = imgData.shape[0:2]    
                
                tile[fpMask] = imgData[(ny*(decArr - fpMinDec)/(fpMaxDec - fpMinDec)).astype(int)[fpMask],
                                       (nx*(1-(raArr - fpMinRa)/(fpMaxRa - fpMinRa))).astype(int)[fpMask]]
                                
        # ^ THIS IS ALL TEMPORARY ^

        
        
        return tile
    
    return vec2Pix

In [ ]:
sampler = normalizer(M101Sampler(),-3,10)
output = 'toasts/ps1_M101'
depth = 8  
toast(sampler, depth, output, ra_range=[205,215],dec_range=[50,60])

In [ ]:
sampler = 'toasts/ps1_bottom'
output = 'toasts/ps1_bottom'
depth = 4 
toast(sampler, depth, output)

In [46]:
Data = namedtuple('Data','img cnt')

class fitsCache:
    """"Caching for fitfile image data"""
    def __init__(self,maxItems):
        self.cache = {}
        self.count = 0
        self.maxItms = maxItems
        
    def remove(self,filename):
        if filename not in self.cache:
            return
        del(self.cache[filename])
    
    def oldest(self):
        return sorted([(y.cnt,x) for x,y in seltf.cache.items()])[0][1]
    
    def add(self,filename):
        if len(self.cache) == self.maxItms:
            self.remove(self.oldest())
        fitsfile = fits.open(filename)
        self.cache[filename] = Data(img=fitsfile[1].data,cnt=self.count)
        fitsfile.close()
        self.count += 1

    def get(self,filename):
        if filename not in self.cache:
            self.add(filename)
        return self.cache[filename].img # MAKE SURE THIS IS NOT MAKING A COPY OF THE DATA
    

In [47]:
psCache = fitsCache(10)

In [48]:
def panstarrsSampler():
    
    psCells = ascii.read("filter_r_rings.rpt")
    psSC2FileLoc = np.full((max(psCells['SCn']) + 1,max(psCells['SCm']) + 1),"",dtype='<U168')
    #psSC2FileLoc[psCells['SCn'],psCells['SCm']] = psCells['fileNPath']
    psSC2FileLoc[2404][5]  = "RINGS.V3.skycell.2404.005.stk.4032678.unconv.fits"
    psSC2FileLoc[2381][62] = "RINGS.V3.skycell.2381.062.stk.3906396.unconv.fits"
    psSC2FileLoc[2381][63] = "RINGS.V3.skycell.2381.063.stk.3906397.unconv.fits"
    psSC2FileLoc[2381][64] = "RINGS.V3.skycell.2381.064.stk.3906398.unconv.fits"
    psSC2FileLoc[2381][52] = "RINGS.V3.skycell.2381.052.stk.3906388.unconv.fits"
    psSC2FileLoc[2381][53] = "rings.v3.skycell.2381.053.stk.r.unconv.fits" # M101
    psSC2FileLoc[2381][54] = "RINGS.V3.skycell.2381.054.stk.3906390.unconv.fits"
    psSC2FileLoc[2381][42] = "RINGS.V3.skycell.2381.042.stk.3906380.unconv.fits"
    psSC2FileLoc[2381][43] = "RINGS.V3.skycell.2381.043.stk.3906381.unconv.fits"
    psSC2FileLoc[2381][44] = "RINGS.V3.skycell.2381.044.stk.3906382.unconv.fits"
                        
    def vec2Pix(raArr,decArr):
        
        global psCache
        
        raArr[raArr < 0] += 2 * np.pi # making all ras 0 - 2pi
        
        # Getting info about skycell and pixel location for given ra/decs
        pixelInfoArray = pssc.findskycell(np.rad2deg(raArr), np.rad2deg(decArr))
            
        # Getting the file paths and creating a unique list of them
        fileLocByPix = psSC2FileLoc[pixelInfoArray['projcell'],pixelInfoArray['subcell']] 
        filePths = np.unique(fileLocByPix)
        
        # Getting the pixel indices
        pixIndX = np.rint(pixelInfoArray['x']).astype(int)
        pixIndY = np.rint(pixelInfoArray['y']).astype(int)
        
        
        tile = np.zeros(raArr.shape) # this should be filled with whatever we want to signal "no data" 
                                     # I am currently using zero
            
        for dataFle in filePths:
            if dataFle == '':
                #print("File not available")
                continue
            
            #print("File location:",dataFle)
            
            # getting the image data (will be rplaced by cache class)
            #fitsFile = fits.open(dataFle)
            #imgData = fitsFile[1].data
            #fitsFile.close()   
            imgData = psCache.get(dataFle)
                
            # getting the pixels we want out of this file
            pix2Fill = (fileLocByPix == dataFle)
            #print("Number of pixels to be filled:",np.sum(pix2Fill))
            tile[pix2Fill] = imgData[pixIndY[pix2Fill],pixIndX[pix2Fill]]   
            
        return tile   
        
    return vec2Pix

In [49]:
start = time.time()
sampler = normalizer(panstarrsSampler(),-3,10)
output = 'toasts/ps1_M101_new'
depth = 8  
toast(sampler, depth, output, ra_range=[210.3,211],dec_range=[54,55.6])
end = time.time()

Exception ignored in: <bound method FITS_rec.__del__ of FITS_rec([(array([101,   0], dtype=int32),),
       (array([11133,   101], dtype=int32),),
       (array([11136, 11234], dtype=int32),), ...,
       (array([   11320, 68463836], dtype=int32),),
       (array([   11328, 68475156], dtype=int32),),
       (array([     101, 68486484], dtype=int32),)], 
      dtype=(numpy.record, [('COMPRESSED_DATA', '>i4', (2,))]))>
Traceback (most recent call last):
  File "/Users/cbrasseur/anaconda/envs/toasty/lib/python3.5/site-packages/astropy/io/fits/fitsrec.py", line 640, in __del__
    del self._coldefs
  File "/Users/cbrasseur/anaconda/envs/toasty/lib/python3.5/site-packages/astropy/io/fits/fitsrec.py", line 637, in _coldefs
    raise AttributeError(exc.args[0])
AttributeError: _coldefs
Exception ignored in: <bound method FITS_rec.__del__ of FITS_rec([(array([101,   0], dtype=int32),),
       (array([10173,   101], dtype=int32),),
       (array([10174, 10274], dtype=int32),), ...,
       (arra

In [51]:
start = time.time()
sampler = normalizer(panstarrsSampler(),-3,10)
output = 'toasts/ps1_M101_new'
depth = 12  
toast(sampler, depth, output, ra_range=[210.589,211],dec_range=[54.26,54.45])
end = time.time()

In [52]:
end-start

226.30935192108154